In [5]:
%load_ext sql
%sql sqlite:///chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @chinook.db'

Проверяем, что версия корректна

In [11]:
%%sql
select SQLITE_VERSION() 

 * sqlite:///chinook.db
Done.


SQLITE_VERSION()
3.26.0


## Задание 1
Для каждого артиста вывести все жанры, которые есть в его песнях, и для каждого жанра вывести наиболее продолжительную песню

In [12]:
%%sql

SELECT Artists, Genre, TrackName, Duration
FROM (
        SELECT ArtistID, Artists, Genre, TrackName, Duration,  row_number() OVER(PARTITION BY ArtistID, Artists, Genre ORDER BY Duration DeSC) as rn
        FROM (
            SELECT artists.artistid as ArtistID, artists.name as Artists, genres.name as Genre, 
                tracks.name as TrackName, tracks.milliseconds as Duration
            FROM tracks
            JOIN genres USING(genreid)
            JOIN albums USING(albumid)
            JOIN artists USING(artistid)
            )
        GROUP BY ArtistID, Artists, Genre, TrackName
    )
WHERE rn = 1
limit 10

 * sqlite:///chinook.db
Done.


Artists,Genre,TrackName,Duration
AC/DC,Rock,Overdose,369319
Accept,Rock,Princess of the Dawn,375418
Aerosmith,Rock,Livin' On The Edge,381231
Alanis Morissette,Rock,You Oughta Know (Alternate),491885
Alice In Chains,Rock,"Love, Hate, Love",387134
Antônio Carlos Jobim,Jazz,O Boto (Bôto),366837
Antônio Carlos Jobim,Latin,Só Tinha De Ser Com Você,389642
Apocalyptica,Metal,Master Of Puppets,436453
Audioslave,Alternative,Show Me How to Live (Live at the Quart Festival),301974
Audioslave,Alternative & Punk,The Curse,309786


## Задание 2 
Для каждой компании вывести трек, за который компания заплатила наибольшую сумму, вывести жанр данного трека и его альбом

In [3]:
%%sql
WITH tab1 as (
    SELECT invoice_items.TrackId, customers.Company, SUM(invoice_items.UnitPrice * invoice_items.Quantity) AS trackCost
    FROM invoice_items
    JOIN invoices USING(InvoiceId)
    JOIN customers USING(CustomerId)
    GROUP BY invoice_items.InvoiceLineId, invoice_items.UnitPrice, invoice_items.Quantity
    HAVING customers.Company IS NOT NULL
    ),

tab2 as (
    SELECT Company, SUM(trackCost) AS sumOfCost
    FROM tab1
    GROUP BY Company, TrackId, trackCost
    ),

tab3 as (
    SELECT Company, MAX(sumOfCost) AS maxCost
    FROM tab2
    GROUP BY Company
    ),

tab4 as (
    SELECT tracks.Name as Track, albums.Title as AlbumTitle, genres.Name as Genre, tab3.Company as Company,
    row_number() OVER (PARTITION BY tab3.Company) as rn
    FROM tracks
    JOIN invoice_items USING(trackid)
    JOIN invoices USING(InvoiceId)
    JOIN customers USING(CustomerId)
    JOIN albums USING(AlbumId)
    JOIN genres USING(GenreId)
    JOIN tab3 on tab3.Company  = customers.Company 
    WHERE tab3.maxCost = tracks.UnitPrice
    )

SELECT Track, AlbumTitle, Genre, Company
FROM tab4
WHERE rn = 1

 * sqlite:///chinook.db
Done.


Track,AlbumTitle,Genre,Company
Gay Witch Hunt,"The Office, Season 3",TV Shows,Apple Inc.
Leper Messiah,Master Of Puppets,Metal,Banco do Brasil S.A.
Experiment In Terra,"Battlestar Galactica (Classic), Season 1",Sci Fi & Fantasy,Embraer - Empresa Brasileira de Aeronáutica S.A.
Valentino's,Heart of the Night,Jazz,Google Inc.
The Hard Part,"Heroes, Season 1",Drama,JetBrains s.r.o.
"Exodus, Pt. 2","Battlestar Galactica, Season 3",TV Shows,Microsoft Corporation
Right Next Door to Hell,Use Your Illusion I,Rock,Riotur
The Beginning of the End,"LOST, Season 4",Drama,Rogers Canada
Right Through You,Jagged Little Pill,Rock,Telus
Admirável Gado Novo,Cássia Eller - Sem Limite [Disc 1],Latin,Woodstock Discos


## Задание 3
Вывести все интервалы, в которые не было нанято ни одного сотрудника. Минимальная дата - 1999-01-01, Максимальная - 2020-01-01.
Работать с таблицей employees, c hiredate

In [4]:
%%sql
WITH tab1 as(
    SELECT DISTINCT  HireDate 
    FROM employees
    WHERE HireDate > '1999-01-01' AND HireDate < '2020-01-01'
),

tab2 as(
    SELECT LAG(HireDate, 1,'1999-01-01 00:00:00') OVER(ORDER BY HireDate) as StartDate, HireDate
    FROM tab1
),

tab3 as(
    SELECT HireDate, LEAD(HireDate, 1,'2020-01-01 00:00:00') OVER(ORDER BY HireDate) StartDate
    FROM tab1
)

SELECT StartDate, HireDate
FROM tab2
UNION
SELECT HireDate, StartDate
FROM tab3

 * sqlite:///chinook.db
Done.


StartDate,HireDate
1999-01-01 00:00:00,2002-04-01 00:00:00
2002-04-01 00:00:00,2002-05-01 00:00:00
2002-05-01 00:00:00,2002-08-14 00:00:00
2002-08-14 00:00:00,2003-05-03 00:00:00
2003-05-03 00:00:00,2003-10-17 00:00:00
2003-10-17 00:00:00,2004-01-02 00:00:00
2004-01-02 00:00:00,2004-03-04 00:00:00
2004-03-04 00:00:00,2020-01-01 00:00:00


## Задание 4
Для каждого плейлиста вывести трек с наибольшей продолжительностью, трек с наименьшей стоимостью (если у треков одинаковая стоимость, то вывести тот, который весит меньше всех), и жанр данного плейлиста (жанром плейлиста называется тот жанр, треков которого больше всех в плейлисте).

In [5]:
%%sql
WITH tab1 as(
    SELECT playlists.name as title, genres.name, playlists.PlaylistId, COUNT(*) as rowCount
    FROM tracks 
    JOIN genres USING(GenreId)    
    JOIN playlist_track USING(TrackId) 
    JOIN playlists USING(PlaylistId)
    JOIN albums USING(AlbumId)
    GROUP BY playlists.PlaylistId, playlists.Name, genres.genreid, genres.name
    ),

tab2 as(
    SELECT name, playlistId
    FROM(
        SELECT title, name, playlistId, row_number() OVER (PARTITION BY PlaylistId ORDER BY RowCount DESC) as rn
        FROM tab1
        )
    WHERE rn = 1
    ),
    
tab3 as(
    SELECT playlist_track.PlaylistId, MAX(Milliseconds) as duration, MIN(UnitPrice) as price, MIN(Bytes) as minBytes
    FROM tracks 
    JOIN genres USING(GenreID)
    JOIN playlist_track USING(TrackId)
    JOIN playlists USING(PlaylistId)
    JOIN albums USING(AlbumId)
    GROUP BY playlist_track.PlaylistId
    ),

tab4 as(
    SELECT playlists.name as PlaylistName, tab2.name as Genre, tracks.Name as Track, tab3.duration, 
        tab3.price as Cost, tab3.minBytes as Byte,
        row_number() OVER (PARTITION BY tab3.PlaylistId ORDER BY tracks.Milliseconds DESC) AS rnMilliseconds,
        row_number() OVER (PARTITION BY tab3.PlaylistId ORDER BY tracks.Bytes ASC) AS rnBytes
    FROM tab2
    JOIN tab3 USING(PlaylistId)
    JOIN playlists USING(PlaylistId)
    JOIN playlist_track USING(PlaylistId)
    JOIN tracks USING(TrackId)
    WHERE tab3.price = tracks.UnitPrice OR tab3.duration = tracks.Milliseconds
    )


SELECT genre, PlaylistName, track, duration, byte, cost
FROM tab4
WHERE rnMilliseconds = 1 OR rnBytes = 1

 * sqlite:///chinook.db
Done.


Genre,PlaylistName,Track,duration,Byte,Cost
Rock,Music,Dazed And Confused,1612329,38747,0.99
Rock,Music,É Uma Partida De Futebol,1612329,38747,0.99
TV Shows,TV Shows,Occupation / Precipice,5286953,20831818,1.99
TV Shows,TV Shows,LOST Season 4 Trailer,5286953,20831818,1.99
Rock,90’s Music,Dazed And Confused,1116734,38747,0.99
Rock,90’s Music,É Uma Partida De Futebol,1116734,38747,0.99
Rock,Music,Dazed And Confused,1612329,38747,0.99
Rock,Music,É Uma Partida De Futebol,1612329,38747,0.99
Alternative,Music Videos,"Band Members Discuss Tracks from ""Revelations""",294294,61118891,0.99
TV Shows,TV Shows,Occupation / Precipice,5286953,20831818,1.99


## Задание 5

Для каждого жанра вывести треки, которые встречались в большем числе заказов, 
сумму, потраченную, на данные треки, количество купленных треков и компанию, которая больше всех приобрела данную композицию

In [49]:
%%sql
WITH tab1 as (
        SELECT invoice_items.TrackId, genres.name as Genre, COUNT(invoice_items.TrackId) as trackCount, 
            SUM(Quantity) as salesCount, SUM(invoice_items.UnitPrice * invoice_items.Quantity) as trackTotalPrice
        FROM invoice_items
        JOIN tracks USING(TrackId)
        JOIN invoices USING(InvoiceId)
        JOIN customers USING(CustomerId)
        JOIN genres USING(GenreId)
        GROUP BY genres.name, invoice_items.TrackId
        ),

tab2 as (
        SELECT *
        FROM (
            SELECT tab1.Genre, tab1.trackCount, tab1.salesCount, tab1.trackTotalPrice, 
                tab1.TrackId, customers.Company AS Company, tracks.Name AS TrackName, 
                RANK() OVER (PARTITION BY tab1.Genre ORDER BY tab1.trackCount DESC) AS r,
                SUM(tab1.trackTotalPrice) OVER(PARTITION BY tab1.Genre) genreSum
            FROM invoice_items         
            JOIN tracks USING(TrackId)
            JOIN invoices USING(InvoiceId)
            JOIN customers USING(CustomerId)
            JOIN genres USING(GenreId)
            JOIN tab1 USING(TrackId)
            )
        WHERE r = 1
        ),   
    
tab3 as(
    SELECT *, COUNT(*) as Number -- здесь немного улучшить
    FROM tab2
    GROUP BY TrackId, Company
    )

SELECT Genre, TrackName, trackCount, genreSum, salesCount, Company
FROM(
    SELECT *, row_number() OVER (PARTITION BY Genre ORDER BY Number DESC)
    FROM tab3
    )
limit 5

 * sqlite:///chinook.db
Done.


Genre,TrackName,trackCount,genreSum,salesCount,Company
Alternative,Say Hello 2 Heaven,1,13.860000000000001,1,None
Alternative,Call Me a Dog,1,13.860000000000001,1,None
Alternative,Four Walled World,1,13.860000000000001,1,None
Alternative,All Night Thing,1,13.860000000000001,1,None
Alternative,Safe and Sound,1,13.860000000000001,1,None
